In [41]:
# import

import math
import pymysql 
import requests
from urllib.parse import unquote, urlencode, quote_plus 
from datetime import datetime, date, timedelta

In [43]:
# 기상청 공개 위도 경도 xy 좌표

NX = 149            ## X축 격자점 수
NY = 253            ## Y축 격자점 수

Re = 6371.00877     ##  지도반경
grid = 5.0          ##  격자간격 (km)
slat1 = 30.0        ##  표준위도 1
slat2 = 60.0        ##  표준위도 2
olon = 126.0        ##  기준점 경도
olat = 38.0         ##  기준점 위도
xo = 210 / grid     ##  기준점 X좌표
yo = 675 / grid     ##  기준점 Y좌표
first = 0

if first == 0 :
    PI = math.asin(1.0) * 2.0
    DEGRAD = PI/ 180.0
    RADDEG = 180.0 / PI

    re = Re / grid
    slat1 = slat1 * DEGRAD
    slat2 = slat2 * DEGRAD
    olon = olon * DEGRAD
    olat = olat * DEGRAD

    sn = math.tan(PI * 0.25 + slat2 * 0.5) / math.tan(PI * 0.25 + slat1 * 0.5)
    sn = math.log(math.cos(slat1) / math.cos(slat2)) / math.log(sn)
    sf = math.tan(PI * 0.25 + slat1 * 0.5)
    sf = math.pow(sf, sn) * math.cos(slat1) / sn
    ro = math.tan(PI * 0.25 + olat * 0.5)
    ro = re * sf / math.pow(ro, sn)
    first = 1

def mapToGrid(lat, lon, code = 0 ):
    ra = math.tan(PI * 0.25 + lat * DEGRAD * 0.5)
    ra = re * sf / pow(ra, sn)
    theta = lon * DEGRAD - olon
    if theta > PI :
        theta -= 2.0 * PI
    if theta < -PI :
        theta += 2.0 * PI
    theta *= sn
    x = (ra * math.sin(theta)) + xo
    y = (ro - ra * math.cos(theta)) + yo
    x = int(x + 1.5)
    y = int(y + 1.5)
    return x, y

def gridToMap(x, y, code = 1):
    x = x - 1
    y = y - 1
    xn = x - xo
    yn = ro - y + yo
    ra = math.sqrt(xn * xn + yn * yn)
    if sn < 0.0 :
        ra = -ra
    alat = math.pow((re * sf / ra), (1.0 / sn))
    alat = 2.0 * math.atan(alat) - PI * 0.5
    if math.fabs(xn) <= 0.0 :
        theta = 0.0
    else :
        if math.fabs(yn) <= 0.0 :
            theta = PI * 0.5
            if xn < 0.0 :
                theta = -theta
        else :
            theta = math.atan2(xn, yn)
    alon = theta / sn + olon
    lat = alat * RADDEG
    lon = alon * RADDEG

    return lat, lon

nx, ny = mapToGrid(35.191619912401606, 129.09438859263133)

In [44]:
url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"

serviceKey = "wGokgRxD1t3z5G4u7MsWumpoCeiWO8JM6yZ87rX1ELTO9nMSUuMOQjHj70rAzuopgyB1iLdKX0S9WK0RLs88bQ==" # 공공데이터 포털에서 생성된 본인의 서비스 키를 복사 / 붙여넣기
serviceKeyDecoded = unquote(serviceKey, 'UTF-8') # 공데이터 포털에서 제공하는 서비스키는 이미 인코딩된 상태이므로, 디코딩하여 사용해야 함



API_TIME = [2, 5, 8, 11, 14, 17, 20, 23]
API_MINUTE = 10

now = datetime.now()

for time in range(7):
    if now.hour is API_TIME[time] and now.minute >= API_MINUTE:
        break 
    elif API_TIME[time - 1] < now.hour <= API_TIME[time]:
        base_date = datetime.today().strftime("%Y%m%d")
        print(base_date)
        base_time = str(API_TIME[time-1]) + str(API_MINUTE)
        print(base_time)
    else:
        time += 1

20230320
1410


In [46]:
queryParams = '?' + urlencode({ quote_plus('serviceKey') : serviceKeyDecoded, quote_plus('base_date') : base_date, 
                                    quote_plus('base_time') : base_time, quote_plus('nx') : nx, quote_plus('ny') : ny,
                                    quote_plus('dataType') : 'json', quote_plus('numOfRows') : '700'}) #페이지로 안나누고 한번에 받아오기 위해 numOfRows=60으로 설정해주었다
                                   

    # 값 요청 (웹 브라우저 서버에서 요청 - url주소와 파라미터)
res = requests.get(url + queryParams, verify=False) 
items = res.json().get('response').get('body').get('items')
items

c:\DEV\Langs\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'item': [{'baseDate': '20230320',
   'baseTime': '1400',
   'category': 'TMP',
   'fcstDate': '20230320',
   'fcstTime': '1500',
   'fcstValue': '17',
   'nx': 98,
   'ny': 76},
  {'baseDate': '20230320',
   'baseTime': '1400',
   'category': 'UUU',
   'fcstDate': '20230320',
   'fcstTime': '1500',
   'fcstValue': '2',
   'nx': 98,
   'ny': 76},
  {'baseDate': '20230320',
   'baseTime': '1400',
   'category': 'VVV',
   'fcstDate': '20230320',
   'fcstTime': '1500',
   'fcstValue': '3.4',
   'nx': 98,
   'ny': 76},
  {'baseDate': '20230320',
   'baseTime': '1400',
   'category': 'VEC',
   'fcstDate': '20230320',
   'fcstTime': '1500',
   'fcstValue': '210',
   'nx': 98,
   'ny': 76},
  {'baseDate': '20230320',
   'baseTime': '1400',
   'category': 'WSD',
   'fcstDate': '20230320',
   'fcstTime': '1500',
   'fcstValue': '3.9',
   'nx': 98,
   'ny': 76},
  {'baseDate': '20230320',
   'baseTime': '1400',
   'category': 'SKY',
   'fcstDate': '20230320',
   'fcstTime': '1500',
   'fcstValue

In [47]:
weather_data = dict()

for item in items['item']:
    # 날짜/시간 확인
    Date = item['fcstDate']
    Time = item['fcstTime']
    if item['fcstDate'] not in weather_data:
        weather_data[Date] = dict()
    if item['fcstTime'] not in weather_data[Date]:
        weather_data[Date][Time] = dict()
 
    # 예보 값
    # 강수확률 (%)
    if item['category'] == 'POP':
        weather_data[Date][Time]['pop'] = item['fcstValue']
    # 강수형태 
    if item['category'] == 'PTY':
        weather_data[Date][Time]['pty'] = item['fcstValue']
    # 1시간 강수량
    if item['category'] == 'PCP':
        weather_data[Date][Time]['pcp'] = item['fcstValue']
    # 습도
    if item['category'] == 'REH':
        weather_data[Date][Time]['reh'] = item['fcstValue']
    # 1시간 신적설
    if item['category'] == 'SNO':
        weather_data[Date][Time]['sno'] = item['fcstValue']
    # 하늘상태
    if item['category'] == 'SKY':
        weather_data[Date][Time]['sky'] = item['fcstValue']
    # 1시간 기온
    if item['category'] == 'TMP':
        weather_data[Date][Time]['tmp'] = item['fcstValue']
    # 일 최저기온
    if item['category'] == 'TMN':
        weather_data[Date][Time]['tmn'] = item['fcstValue']
    # 일 최고기온
    if item['category'] == 'TMX':
        weather_data[Date][Time]['tmx'] = item['fcstValue']
    # 풍속(동서성분)
    if item['category'] == 'UUU':
        weather_data[Date][Time]['uuu'] = item['fcstValue']
    # 풍속(남북성분)
    if item['category'] == 'VVV':
        weather_data[Date][Time]['vvv'] = item['fcstValue']
    # 파고
    if item['category'] == 'WAV':
        weather_data[Date][Time]['wav'] = item['fcstValue']
    # 풍향
    if item['category'] == 'VEC':
        weather_data[Date][Time]['vec'] = item['fcstValue']
    # 풍속
    if item['category'] == 'WSD':
        weather_data[Date][Time]['wsd'] = item['fcstValue']
    
print(weather_data)

{'20230320': {'1500': {'tmp': '17', 'uuu': '2', 'vvv': '3.4', 'vec': '210', 'wsd': '3.9', 'sky': '1', 'pty': '0', 'pop': '0', 'wav': '0', 'pcp': '강수없음', 'reh': '45', 'sno': '적설없음'}, '1600': {'tmp': '16', 'uuu': '2.4', 'vvv': '3.1', 'vec': '218', 'wsd': '3.9', 'sky': '1', 'pty': '0', 'pop': '0', 'wav': '0', 'pcp': '강수없음', 'reh': '45', 'sno': '적설없음'}, '1700': {'tmp': '16', 'uuu': '2.7', 'vvv': '2.7', 'vec': '225', 'wsd': '3.8', 'sky': '1', 'pty': '0', 'pop': '0', 'wav': '0', 'pcp': '강수없음', 'reh': '45', 'sno': '적설없음'}, '1800': {'tmp': '15', 'uuu': '1.8', 'vvv': '1.8', 'vec': '225', 'wsd': '2.5', 'sky': '1', 'pty': '0', 'pop': '0', 'wav': '0', 'pcp': '강수없음', 'reh': '50', 'sno': '적설없음'}, '1900': {'tmp': '13', 'uuu': '1.3', 'vvv': '1.4', 'vec': '223', 'wsd': '1.9', 'sky': '1', 'pty': '0', 'pop': '0', 'wav': '0', 'pcp': '강수없음', 'reh': '60', 'sno': '적설없음'}, '2000': {'tmp': '13', 'uuu': '1.4', 'vvv': '1.2', 'vec': '229', 'wsd': '1.8', 'sky': '1', 'pty': '0', 'pop': '0', 'wav': '0', 'pcp': '강수없음

In [48]:
conn = pymysql.connect(host='210.119.12.66', user = 'root', password='12345',
                           db = 'miniproject01', charset='utf8')
cur = conn.cursor()

day = '20230320'
time = '1200'

sql = 'SELECT MAX(idx) FROM ohyurina' 
cur.execute(sql)
rows = cur.fetchall()
conn.close()

idx = rows[0][0] + 1

In [49]:
conn = pymysql.connect(host='210.119.12.66', user = 'root', password='12345',
                           db = 'miniproject01', charset='utf8')
cur = conn.cursor()
sql = 'SELECT * FROM ohyurina WHERE log_date = \'20230317\' and log_time = \'1400\'' 
cur.execute(sql)
rows = cur.fetchall()
conn.close()

In [50]:
for day in weather_data:
    for time in weather_data[day]:
        conn = pymysql.connect(host='210.119.12.66', user = 'root', password='12345',
                           db = 'miniproject01', charset='utf8')
        cur = conn.cursor()
        sql = 'SELECT * FROM ohyurina WHERE log_date = %s and log_time = %s' 
        cur.execute(sql, (day, time))
        rows = cur.fetchall()
        conn.close()
        
        if bool(rows):
            conn = pymysql.connect(host='210.119.12.66', user = 'root', password='12345',
                           db = 'miniproject01', charset='utf8')
            cur = conn.cursor()
            if 'sno' in weather_data[day][time].keys():
                if 'tmn' in weather_data[day][time].keys():
                    query = '''UPDATE ohyurina SET
                            tmp = %s, uuu = %s, vvv = %s,
                            vec = %s, wsd = %s, sky = %s,
                            pty = %s, pop = %s, wav = %s, 
                            pcp = %s, reh = %s, sno = %s, tmn = %s
                            WHERE log_date = %s and log_time = %s
                            '''
                    cur.execute(query, (weather_data[day][time]['tmp'], weather_data[day][time]['uuu'], weather_data[day][time]['vvv'], weather_data[day][time]['vec'],
                                weather_data[day][time]['wsd'], weather_data[day][time]['sky'], weather_data[day][time]['pty'], weather_data[day][time]['pop'],
                                weather_data[day][time]['wav'], weather_data[day][time]['pcp'], weather_data[day][time]['reh'], weather_data[day][time]['sno'],
                                weather_data[day][time]['tmn'], day, time)
                                )
                elif 'tmx' in weather_data[day][time].keys():
                    query = '''UPDATE ohyurina SET
                            tmp = %s, uuu = %s, vvv = %s,
                            vec = %s, wsd = %s, sky = %s,
                            pty = %s, pop = %s, wav = %s, 
                            pcp = %s, reh = %s, sno = %s, tmx = %s
                            WHERE log_date = %s and log_time = %s
                            '''
                    cur.execute(query, (weather_data[day][time]['tmp'], weather_data[day][time]['uuu'], weather_data[day][time]['vvv'], 
                                    weather_data[day][time]['vec'], weather_data[day][time]['wsd'], weather_data[day][time]['sky'], weather_data[day][time]['pty'], weather_data[day][time]['pop'],
                                    weather_data[day][time]['wav'], weather_data[day][time]['pcp'], weather_data[day][time]['reh'], weather_data[day][time]['sno'], weather_data[day][time]['tmx'], day, time))
                else:
                    query = '''UPDATE ohyurina SET
                            tmp = %s, uuu = %s, vvv = %s,
                            vec = %s, wsd = %s, sky = %s,
                            pty = %s, pop = %s, wav = %s, 
                            pcp = %s, reh = %s, sno = %s
                            WHERE log_date = %s and log_time = %s
                            '''
                    cur.execute(query, (weather_data[day][time]['tmp'], weather_data[day][time]['uuu'], weather_data[day][time]['vvv'],
                                    weather_data[day][time]['vec'], weather_data[day][time]['wsd'], weather_data[day][time]['sky'], weather_data[day][time]['pty'], weather_data[day][time]['pop'],
                                    weather_data[day][time]['wav'], weather_data[day][time]['pcp'], weather_data[day][time]['reh'], weather_data[day][time]['sno'], day, time))
            else:
                query = '''UPDATE ohyurina SET
                            tmp = %s, uuu = %s, vvv = %s,
                            vec = %s, wsd = %s, sky = %s,
                            pty = %s, pop = %s, wav = %s, 
                            pcp = %s, reh = %s
                            WHERE log_date = %s and log_time = %s
                            '''
                cur.execute(query, (weather_data[day][time]['tmp'], weather_data[day][time]['uuu'], weather_data[day][time]['vvv'],
                                weather_data[day][time]['vec'], weather_data[day][time]['wsd'], weather_data[day][time]['sky'], weather_data[day][time]['pty'], weather_data[day][time]['pop'],
                                weather_data[day][time]['wav'], weather_data[day][time]['pcp'], weather_data[day][time]['reh'], day, time))

        else: 
            conn = pymysql.connect(host='210.119.12.66', user = 'root', password='12345',
                           db = 'miniproject01', charset='utf8')
            cur = conn.cursor()
            if 'sno' in weather_data[day][time].keys():
                if 'tmn' in weather_data[day][time].keys():
                    query = '''INSERT INTO ohyurina (idx, log_date, log_time, tmp, uuu, vvv, vec, wsd, sky, pty, pop, wav, pcp, reh, sno, tmn)
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
                    cur.execute(query, (idx, day, time, weather_data[day][time]['tmp'], weather_data[day][time]['uuu'], weather_data[day][time]['vvv'],
                                    weather_data[day][time]['vec'], weather_data[day][time]['wsd'], weather_data[day][time]['sky'], weather_data[day][time]['pty'], weather_data[day][time]['pop'],
                                    weather_data[day][time]['wav'], weather_data[day][time]['pcp'], weather_data[day][time]['reh'], weather_data[day][time]['sno'], weather_data[day][time]['tmn']))
                elif 'tmx' in weather_data[day][time].keys():
                    query = '''INSERT INTO ohyurina (idx, log_date, log_time, tmp, uuu, vvv, vec, wsd, sky, pty, pop, wav, pcp, reh, sno, tmx)
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
                    cur.execute(query, (idx, day, time, weather_data[day][time]['tmp'], weather_data[day][time]['uuu'], weather_data[day][time]['vvv'],
                                    weather_data[day][time]['vec'], weather_data[day][time]['wsd'], weather_data[day][time]['sky'], weather_data[day][time]['pty'], weather_data[day][time]['pop'],
                                    weather_data[day][time]['wav'], weather_data[day][time]['pcp'], weather_data[day][time]['reh'], weather_data[day][time]['sno'], weather_data[day][time]['tmx']))
                else:
                    query = '''INSERT INTO ohyurina (idx, log_date, log_time, tmp, uuu, vvv, vec, wsd, sky, pty, pop, wav, pcp, reh, sno)
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
                    cur.execute(query, (idx, day, time, weather_data[day][time]['tmp'], weather_data[day][time]['uuu'], weather_data[day][time]['vvv'],
                                    weather_data[day][time]['vec'], weather_data[day][time]['wsd'], weather_data[day][time]['sky'], weather_data[day][time]['pty'], weather_data[day][time]['pop'],
                                    weather_data[day][time]['wav'], weather_data[day][time]['pcp'], weather_data[day][time]['reh'], weather_data[day][time]['sno']))
            else:
                query = '''INSERT INTO ohyurina (idx, log_date, log_time, tmp, uuu, vvv, vec, wsd, sky, pty, pop, wav, pcp, reh)
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
                cur.execute(query, (idx, day, time, weather_data[day][time]['tmp'], weather_data[day][time]['uuu'], weather_data[day][time]['vvv'],
                                weather_data[day][time]['vec'], weather_data[day][time]['wsd'], weather_data[day][time]['sky'], weather_data[day][time]['pty'], weather_data[day][time]['pop'],
                                weather_data[day][time]['wav'], weather_data[day][time]['pcp'], weather_data[day][time]['reh']))

            idx += 1
        conn.commit()

conn.close()
print('저장')

저장
